In [ ]:
# Importing numpy, pandas, math and matplotlib libraries and setting default fonts

import numpy as np
import pandas as pd
import os
import math
import decimal
import matplotlib
import matplotlib.image as image
from matplotlib import artist
from matplotlib import pyplot as plt
from matplotlib import rcParams
import ipywidgets
from ipywidgets import interact, IntSlider, interactive, widgets, interact_manual,HBox,fixed
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')
layout = widgets.Layout(width='400px', height='30px')
pd.options.display.float_format = "{:.2f}".format
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Franklin Gothic Medium', 'Franklin Gothic Book']
%matplotlib inline

In [ ]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
Rotelle = pd.read_csv('Player_Stats_LL.csv', index_col=0) #husk å sette denne til 1 om det ikke funker

Rotelle = Rotelle.fillna(0)

# To display all columns and rows
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None) 

Rotelle['90s'] = Rotelle['minutes']/90

In [ ]:
minutes_position_filtered = (Rotelle['minutes'] >= 90)  & (Rotelle['position'].str.startswith('MF'))
df = Rotelle.loc[minutes_position_filtered, ['player', 'squad', 'minutes', 'position', 'passes_pct', 'progressive_passes', 'passes_into_final_third', 'passes_into_penalty_area', 'assisted_shots','passes_pressure', 'miscontrols', 'dispossessed', 'dribbles_completed_pct', 'dribbles_completed', 'carry_progressive_distance', 'carries', 'carries_into_final_third','touches_att_3rd', 'touches_att_pen_area', 'pass_targets', 'passes_received_pct', 'passes_received', 'gca_per90', 'sca_per90', 'xg_per90', 'npxg_per90','xa_per90', 'npxg_xa_per90', 'xa_net', 'npxg_net','npxg_per_shot', 'shots_on_target_pct', 'aerials_won_pct','aerials_won', 'tackles_won', 'tackles', 'interceptions', 'ball_recoveries', 'pressure_regain_pct', 'dribbled_past', '90s']] #Do not touch this line!!
minutes_filteret = df.set_index('player')

minutes_filteret.head(5)

In [ ]:
passing_percentiles = pd.DataFrame()
passing_percentiles['squad'] = minutes_filteret['squad']

#Passing stats
passing_percentiles['Pass\nCompletion %'] = minutes_filteret['passes_pct'].rank(pct=True)*100
passing_percentiles['Pressured Passes\nPer 90'] = minutes_filteret['passes_pressure'].rank(pct=True)*100
passing_percentiles['Progressive Passes\nPer 90'] = \
    (minutes_filteret['progressive_passes']/minutes_filteret['90s']).rank(pct=True)*100
passing_percentiles['Successful Deliveries\nInto Box Per 90'] = \
    (minutes_filteret['passes_into_penalty_area']/minutes_filteret['90s']).rank(pct=True)*100
passing_percentiles['Key Passes\n Per 90'] = \
    (minutes_filteret['assisted_shots']/minutes_filteret['90s']).rank(pct=True)*100


#Possesion stats
passing_percentiles['Rate Adj Target of an\nAttempted Pass'] =\
    (minutes_filteret['passes_received']*minutes_filteret['passes_received_pct']).rank(pct=True)*100     
passing_percentiles['Turnovers\nPer 90'] = \
    ((minutes_filteret['miscontrols']+minutes_filteret['dispossessed'])/minutes_filteret['90s']).rank(pct=True, ascending=False)*100
passing_percentiles['Rate Adj Successful\nDribbles Per 90'] =\
    (minutes_filteret['dribbles_completed']*minutes_filteret['dribbles_completed_pct']/ \
    minutes_filteret['90s']).rank(pct=True)*100                                          
passing_percentiles['Progressive Distance\nPer Carry'] = \
    (minutes_filteret['carry_progressive_distance']/minutes_filteret['carries']).rank(pct=True)*100
passing_percentiles['Final Third\nEntries'] = minutes_filteret['carries_into_final_third'].rank(pct=True)*100
passing_percentiles['Touches in\nBox'] = minutes_filteret['touches_att_pen_area'].rank(pct=True)*100


#Contribution stats
passing_percentiles['GCA\nPer 90'] = minutes_filteret['gca_per90'].rank(pct=True)*100
passing_percentiles['SCA\nPer 90'] = minutes_filteret['sca_per90'].rank(pct=True)*100
passing_percentiles['xA\nPer 90'] = minutes_filteret['xa_per90'].rank(pct=True)*100
passing_percentiles['Non-Penalty\nxG Per 90'] = minutes_filteret['npxg_per90'].rank(pct=True)*100
passing_percentiles['Non-Penalty\nxA + xG Per 90'] = minutes_filteret['npxg_xa_per90'].rank(pct=True)*100 
passing_percentiles['xG\nPer Shot'] = minutes_filteret['npxg_per_shot'].rank(pct=True)*100
passing_percentiles['Netto xA'] = minutes_filteret['xa_net'].rank(pct=True)*100 


#Defening stats
passing_percentiles['Rate Adj\nAerials Won %'] = \
    (minutes_filteret['aerials_won']*minutes_filteret['aerials_won_pct']).rank(pct=True)*100     
passing_percentiles['Ball recoveries\nPer 90'] = \
    (minutes_filteret['ball_recoveries']/minutes_filteret['90s']).rank(pct=True)*100    
passing_percentiles['Interceptions\nPer 90'] = \
    (minutes_filteret['interceptions']/minutes_filteret['90s']).rank(pct=True)*100 
passing_percentiles['Pressure\nRegain %'] = minutes_filteret['pressure_regain_pct'].rank(pct=True)*100
passing_percentiles['Rate Adj\n Tackles Won %'] = \
    ((minutes_filteret['tackles_won'])*(minutes_filteret['tackles_won']/minutes_filteret['tackles'])).rank(pct=True)*100
passing_percentiles['Dribbled\nPast'] = minutes_filteret['dribbled_past'].rank(pct=True, ascending=False)*100
                                          
passing_percentiles['minutes'] = minutes_filteret['minutes']
passing_percentiles['position'] = minutes_filteret['position']
passing_percentiles['90s'] = minutes_filteret['90s']

In [ ]:
passing_percentiles['minutes'] = passing_percentiles['minutes'].astype(int)
passing_percentiles['minutes'] = passing_percentiles['minutes'].astype(str)
passing_percentiles = passing_percentiles.fillna(0)
passing_percentiles.head(5)

In [ ]:
def label_adjuster(i):
    
    if i <= 50: #Everything under this percentile will be pushed to a 100 - outside the chart
        return 101
    else:
        return -10

In [ ]:
def percentile_chart(player_name):
    
    # Creates bars based on the player entered
    theta = list(map((lambda x: x*math.pi/24), list(range(1,49,2))))
    radii = list(passing_percentiles.loc[player_name][1:25])
    width = [math.pi/12]*len(radii)
    
    # Aesthetic details of the figure including color, axis tick size, x value specifics
    # and r-axis limits, etc.
    plt.figure(figsize=(10,10), facecolor = '#37474f')
    ax = plt.subplot(111, projection='polar')
    ax.set_facecolor('#37474f')
    #ax = plt.subplot(111, projection='polar')
    ax.grid(True)
    ax.xaxis.grid(linewidth=2.5, color='xkcd:white')
    ax.yaxis.grid(linewidth=1.5, color='xkcd:white', alpha=0.3)
    ax.tick_params(axis='x', colors='#37474f')
    ax.tick_params(axis='y', colors='#37474f')
    ax.set_rlim(0,100)
    ax.spines['polar'].set_color('xkcd:white')
    ax.spines['polar'].set_linewidth(3)
    ax.set_rlabel_position(0)
    ax.set_rticks([25,50,75])
    ax.tick_params(labelsize=0)
    labels = plt.thetagrids(range(0, 360, 15), ('','','','','','','','','','','','','','','','','','','','','','','',''))
    ax.spines['polar'].set_color('xkcd:white')
    ax.spines['polar'].set_linewidth(2.5)
    ax.set_rorigin(-15)
   
    # Aesthetic details of the bars and chart titles based on player name
    ax.bar(theta, radii, width, color=['#ffba08','#ffba08','#ffba08','#ffba08','#ffba08','#219ebc','#219ebc','#219ebc','#219ebc','#219ebc','#219ebc','#ef233c','#ef233c','#ef233c','#ef233c','#ef233c','#ef233c','#ef233c','#2a9d8f','#2a9d8f', '#2a9d8f', '#2a9d8f', '#2a9d8f','#2a9d8f'], \
           alpha=0.7, edgecolor='xkcd:white', linewidth=0.5)
    ax.set_title('\n'+player_name+' - Performance Overview\n\n\n', \
                 color='xkcd:white', fontsize=26) 
    plt.text(0.5, 1.13, "Position: " + r"$\bf{" + passing_percentiles.at[player_name, 'position'] + "}$" + " || " + " Minutes Played: " + r"$\bf{" + passing_percentiles.at[player_name, 'minutes'] + "}$" + " || " + " Club: " + r"$\bf{" + passing_percentiles.at[player_name, 'squad'] + "}$", \
         horizontalalignment='center', verticalalignment='top', color='#ef233c', size='17', \
             transform=ax.transAxes)

    l = list(passing_percentiles)
    
    #Adds text to the chart in necessary places
    #plt.text(-0.2, -0.1, 'Data obtained from FBRef.com', \
         #horizontalalignment='left', verticalalignment='top', color='xkcd:black', \
             #transform=ax.transAxes, size='14')
    #plt.text(0.5, 1.22, "Percentile Rankings vs MF and FW in Same League, 2020/21", \
         #horizontalalignment='center', verticalalignment='top', color='#212f45', size='14', \
             #transform=ax.transAxes)
    #plt.text(1.2, -0.075, '\nExcludes players with <1200 minutes', \
        #horizontalalignment='right', verticalalignment='top', color='xkcd:black', \
             #style='italic', transform=ax.transAxes, size='14')

    
    #fixing bars lables
    plt.text(1.1, 0.57, l[1], \
         horizontalalignment='center', verticalalignment='top', color='xkcd:white',\
             transform=ax.transAxes, size='13',)
    plt.text(1.08, 0.7, l[2], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13',)
    plt.text(1, 0.83, l[3], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.91, 0.93, l[4], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.76, 1.00, l[5], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.59, 1.055, l[6], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.40, 1.055, l[7], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.24, 1.02, l[8], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.1, 0.93, l[9], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.01, 0.83, l[10], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.05, 0.7, l[11], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.09, 0.57, l[12], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.08, 0.42, l[13], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(-0.03, 0.29, l[14], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(0.028, 0.16, l[15], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(0.13, 0.06, l[16], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(0.27, -0.02, l[17], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.43, -0.04, l[18], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.58, -0.04, l[19], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.73, -0.02, l[20], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(0.89, 0.063, l[21], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')
    plt.text(1, 0.17, l[22], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(1.07, 0.3, l[23], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13') 
    plt.text(1.09, 0.42, l[24], \
         horizontalalignment='center', verticalalignment='center', color='xkcd:white',\
             transform=ax.transAxes, size='13')    

    
    # Annotating bar labels with their values
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]:
        plt.annotate(str(int(round(radii[i])))+"%", (theta[i], radii[i]+label_adjuster(radii[i])), color='xkcd:white', \
                horizontalalignment='center', fontsize=14)
    
    plt.tight_layout()
    plt.subplots_adjust(left=0.12, right=0.85, top=0.85, bottom=0.08)
    #plt.show() If saving pic this needs to be hashed out

In [ ]:
percentile_chart('Iker Muniain') 

In [ ]:
#Athletic Club

percentile_chart('Álex Berenguer') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Álex Berenguer.png', transparent=True, dpi=72)

percentile_chart('Dani García') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Dani García.png', transparent=True, dpi=72)

percentile_chart('Ibai Gómez') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Ibai Gómez.png', transparent=True, dpi=72)

percentile_chart('Unai López') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Unai López.png', transparent=True, dpi=72)

percentile_chart('Jon Morcillo') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Jon Morcillo.png', transparent=True, dpi=72)

percentile_chart('Iker Muniain') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Iker Muniain.png', transparent=True, dpi=72)

percentile_chart('Unai Vencedor Paris') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Unai Vencedor Paris.png', transparent=True, dpi=72)

percentile_chart('Mikel Vesga') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Mikel Vesga.png', transparent=True, dpi=72)

percentile_chart('Oier Zarraga') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\LL_DataBase\Athletic Club\MF\Oier Zarraga.png', transparent=True, dpi=72)

In [ ]:
#Aston Villa

percentile_chart('Ross Barkley') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\MF\Ross Barkley.png', transparent=True, dpi=72)

percentile_chart('Conor Hourihane') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\MF\Conor Hourihane.png', transparent=True, dpi=72)

percentile_chart('Douglas Luiz') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\MF\Douglas Luiz.png', transparent=True, dpi=72)

percentile_chart('John McGinn') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\MF\John McGinn.png', transparent=True, dpi=72)

percentile_chart('Marvelous Nakamba') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\MF\Marvelous Nakamba.png', transparent=True, dpi=72)

percentile_chart('Jacob Ramsey') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Aston Villa\MF\Jacob Ramsey.png', transparent=True, dpi=72)

In [ ]:
#Brighton

percentile_chart('Steven Alzate') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\MF\Steven Alzate.png', transparent=True, dpi=72)

percentile_chart('Yves Bissouma') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\MF\Yves Bissouma.png', transparent=True, dpi=72)

percentile_chart('Pascal Groß') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\MF\Pascal Groß.png', transparent=True, dpi=72)

percentile_chart('Adam Lallana') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\MF\Adam Lallana.png', transparent=True, dpi=72)

percentile_chart('Alexis Mac Allister') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\MF\Alexis Mac Allister.png', transparent=True, dpi=72)

percentile_chart('Davy Pröpper') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Brighton\MF\Davy Pröpper.png', transparent=True, dpi=72)

In [ ]:
#Burnley

percentile_chart('Josh Benson') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Josh Benson.png', transparent=True, dpi=72)

percentile_chart('Jóhann Berg Guðmundsson') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Jóhann Berg Guðmundsson.png', transparent=True, dpi=72)

percentile_chart('Robbie Brady') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Robbie Brady.png', transparent=True, dpi=72)

percentile_chart('Josh Brownhill') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Josh Brownhill.png', transparent=True, dpi=72)

percentile_chart('Jack Cork') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Jack Cork.png', transparent=True, dpi=72)

percentile_chart('Dwight McNeil') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Dwight McNeil.png', transparent=True, dpi=72)

percentile_chart('Dale Stephens') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Dale Stephens.png', transparent=True, dpi=72)

percentile_chart('Ashley Westwood') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Burnley\MF\Ashley Westwood.png', transparent=True, dpi=72)

In [ ]:
#Chelsea

percentile_chart('Kai Havertz') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\MF\Kai Havertz.png', transparent=True, dpi=72)

percentile_chart('Jorginho') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\MF\Jorginho.png', transparent=True, dpi=72)

percentile_chart("N'Golo Kanté") 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\MF\Kanté.png', transparent=True, dpi=72)

percentile_chart('Mateo Kovačić') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\MF\Mateo Kovačić.png', transparent=True, dpi=72)

percentile_chart('Mason Mount') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Chelsea\MF\Mason Mount.png', transparent=True, dpi=72)

In [ ]:
#Crystal Palace

percentile_chart('Eberechi Eze') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\MF\Eberechi Eze.png', transparent=True, dpi=72)

percentile_chart('James McArthur') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\MF\James McArthur.png', transparent=True, dpi=72)

percentile_chart('Luka Milivojević') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\MF\Luka Milivojević.png', transparent=True, dpi=72)

percentile_chart('Jaïro Riedewald') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\MF\Jaïro Riedewald.png', transparent=True, dpi=72)

percentile_chart('Jeffrey Schlupp') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\MF\Jeffrey Schlupp.png', transparent=True, dpi=72)

percentile_chart('Andros Townsend') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\MF\Andros Townsend.png', transparent=True, dpi=72)

percentile_chart('James McCarthy') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Crystal Palace\MF\James McCarthy.png', transparent=True, dpi=72)

In [ ]:
#Everton

percentile_chart('Allan') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\MF\Allan.png', transparent=True, dpi=72)

percentile_chart('Tom Davies')
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\MF\Tom Davies.png', transparent=True, dpi=72)

percentile_chart('Fabian Delph') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\MF\Fabian Delph.png', transparent=True, dpi=72)

percentile_chart('Abdoulaye Doucouré') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\MF\Abdoulaye Doucouré.png', transparent=True, dpi=72)

percentile_chart('André Gomes') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\MF\André Gomes.png', transparent=True, dpi=72)

percentile_chart('Gylfi Sigurðsson') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Everton\MF\Gylfi Sigurðsson.png', transparent=True, dpi=72)

In [ ]:
#Fulham

percentile_chart('Tom Cairney') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\MF\Tom Cairney.png', transparent=True, dpi=72)

percentile_chart('Mario Lemina') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\MF\Mario Lemina.png', transparent=True, dpi=72)

percentile_chart('Ruben Loftus-Cheek') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\MF\Ruben Loftus-Cheek.png', transparent=True, dpi=72)

percentile_chart('Josh Onomah') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\MF\Josh Onomah.png', transparent=True, dpi=72)

percentile_chart('Harrison Reed') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\MF\Harrison Reed.png', transparent=True, dpi=72)

percentile_chart('Andre-Frank Zambo Anguissa') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\MF\Andre-Frank Zambo Anguissa.png', transparent=True, dpi=72)

percentile_chart('Bobby Reid') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Fulham\FW\Bobby Reid.png', transparent=True, dpi=72)

In [ ]:
#Leeds United

percentile_chart('Ian Carlo Poveda') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Ian Carlo Poveda.png', transparent=True, dpi=72)

percentile_chart('Hélder Costa') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Hélder Costa.png', transparent=True, dpi=72)

percentile_chart('Raphael Dias Belloli') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Raphael Dias Belloli.png', transparent=True, dpi=72)

percentile_chart('Jack Harrison') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Jack Harrison.png', transparent=True, dpi=72)

percentile_chart('Pablo Hernández') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Pablo Hernández.png', transparent=True, dpi=72)

percentile_chart('Mateusz Klich') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Mateusz Klich.png', transparent=True, dpi=72)

percentile_chart('Kalvin Phillips') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Kalvin Phillips.png', transparent=True, dpi=72)

percentile_chart('Tyler Roberts') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Tyler Roberts.png', transparent=True, dpi=72)

percentile_chart('Rodrigo') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Rodrigo.png', transparent=True, dpi=72)

percentile_chart('Jamie Shackleton') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leeds United\MF\Jamie Shackleton.png', transparent=True, dpi=72)

In [ ]:
#Leicester City

percentile_chart('Hamza Choudhury') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\MF\Hamza Choudhury.png', transparent=True, dpi=72)

percentile_chart('James Maddison') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\MF\James Maddison.png', transparent=True, dpi=72)

percentile_chart('Nampalys Mendy') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\MF\Nampalys Mendy.png', transparent=True, dpi=72)

percentile_chart('Wilfred Ndidi') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\MF\Wilfred Ndidi.png', transparent=True, dpi=72)

percentile_chart('Dennis Praet') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\MF\Dennis Praet.png', transparent=True, dpi=72)

percentile_chart('Youri Tielemans') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\MF\Youri Tielemans.png', transparent=True, dpi=72)

percentile_chart('Cengiz Ünder') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\MF\Cengiz Ünder.png', transparent=True, dpi=72)

percentile_chart('Ayoze Pérez') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Leicester\FW\Ayoze Pérez.png', transparent=True, dpi=72)

In [ ]:
#Liverpool

percentile_chart('Thiago Alcántara') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\MF\Thiago Alcántara.png', transparent=True, dpi=72)

percentile_chart('Jordan Henderson') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\MF\Jordan Henderson.png', transparent=True, dpi=72)

percentile_chart('Curtis Jones') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\MF\Curtis Jones.png', transparent=True, dpi=72)

percentile_chart('Naby Keïta') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\MF\Naby Keïta.png', transparent=True, dpi=72)

percentile_chart('James Milner') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\MF\James Milner.png', transparent=True, dpi=72)

percentile_chart('Xherdan Shaqiri') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\MF\Xherdan Shaqiri.png', transparent=True, dpi=72)

percentile_chart('Georginio Wijnaldum') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Liverpool\MF\Georginio Wijnaldum.png', transparent=True, dpi=72)

In [ ]:
#Manchester City

percentile_chart('Kevin De Bruyne') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\MF\Kevin De Bruyne.png', transparent=True, dpi=72)

percentile_chart('Fernandinho') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\MF\Fernandinho.png', transparent=True, dpi=72)

percentile_chart('İlkay Gündoğan') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\MF\İlkay Gündoğan.png', transparent=True, dpi=72)

percentile_chart('Rodri') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\MF\Rodri.png', transparent=True, dpi=72)

percentile_chart('Bernardo Silva') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester City\MF\Bernardo Silva.png', transparent=True, dpi=72)

In [ ]:
#Manchester United

percentile_chart('Donny van de Beek') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\MF\Donny van de Beek.png', transparent=True, dpi=72)

percentile_chart('Bruno Fernandes') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\MF\Bruno Fernandes.png', transparent=True, dpi=72)

percentile_chart('Fred') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\MF\Fred.png', transparent=True, dpi=72)

percentile_chart('Nemanja Matić') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\MF\Nemanja Matić.png', transparent=True, dpi=72)

percentile_chart('Scott McTominay') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\MF\Scott McTominay.png', transparent=True, dpi=72)

percentile_chart('Paul Pogba') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Manchester United\MF\Paul Pogba.png', transparent=True, dpi=72)

In [ ]:
#Newcastle United

percentile_chart('Miguel Almirón') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\MF\Miguel Almirón.png', transparent=True, dpi=72)

percentile_chart('Isaac Hayden') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\MF\Isaac Hayden.png', transparent=True, dpi=72)

percentile_chart('Jeff Hendrick') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\MF\Jeff Hendrick.png', transparent=True, dpi=72)

percentile_chart('Matthew Longstaff') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\MF\Matthew Longstaff.png', transparent=True, dpi=72)

percentile_chart('Sean Longstaff') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\MF\Sean Longstaff.png', transparent=True, dpi=72)

percentile_chart('Jonjo Shelvey') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Newcastle United\MF\Jonjo Shelvey.png', transparent=True, dpi=72)

In [ ]:
#Sheffield United

percentile_chart('Sander Berge') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\MF\Sander Berge.png', transparent=True, dpi=72)

percentile_chart('John Fleck') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\MF\John Fleck.png', transparent=True, dpi=72)

percentile_chart('John Lundstram') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\MF\John Lundstram.png', transparent=True, dpi=72)

percentile_chart('Oliver Norwood') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\MF\Oliver Norwood.png', transparent=True, dpi=72)

percentile_chart('Ben Osborn') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Sheffield United\MF\Ben Osborn.png', transparent=True, dpi=72)

In [ ]:
#Southampton

percentile_chart('Stuart Armstrong') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\Stuart Armstrong.png', transparent=True, dpi=72)

percentile_chart('Ibrahima Diallo') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\Ibrahima Diallo.png', transparent=True, dpi=72)

percentile_chart('Moussa Djenepo') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\Moussa Djenepo.png', transparent=True, dpi=72)

percentile_chart('Nathan Redmond') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\Nathan Redmond.png', transparent=True, dpi=72)

percentile_chart('Oriol Romeu') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\Oriol Romeu.png', transparent=True, dpi=72)

percentile_chart('William Smallbone') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\William Smallbone.png', transparent=True, dpi=72)

percentile_chart('Theo Walcott') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\Theo Walcott.png', transparent=True, dpi=72)

percentile_chart('James Ward-Prowse') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Southampton\MF\James Ward-Prowse.png', transparent=True, dpi=72)

In [ ]:
#Tottenham

percentile_chart('Dele Alli') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\MF\Dele Alli.png', transparent=True, dpi=72)

percentile_chart('Pierre Højbjerg') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\MF\Pierre Højbjerg.png', transparent=True, dpi=72)

percentile_chart('Giovani Lo Celso') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\MF\Giovani Lo Celso.png', transparent=True, dpi=72)

percentile_chart('Tanguy Ndombele') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\MF\Tanguy Ndombele.png', transparent=True, dpi=72)

percentile_chart('Moussa Sissoko') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\MF\Moussa Sissoko.png', transparent=True, dpi=72)

percentile_chart('Harry Winks') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Tottenham\MF\Harry Winks.png', transparent=True, dpi=72)

In [ ]:
#West Brom

percentile_chart('Grady Diangana') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Grady Diangana.png', transparent=True, dpi=72)

percentile_chart('Kyle Edwards') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Kyle Edwards.png', transparent=True, dpi=72)

percentile_chart('Conor Gallagher') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Conor Gallagher.png', transparent=True, dpi=72)

percentile_chart('Filip Krovinović') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Filip Krovinović.png', transparent=True, dpi=72)

percentile_chart('Jake Livermore') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Jake Livermore.png', transparent=True, dpi=72)

percentile_chart('Ainsley Maitland-Niles') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Ainsley Maitland-Niles.png', transparent=True, dpi=72)

percentile_chart('Matheus Pereira') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Matheus Pereira.png', transparent=True, dpi=72)

percentile_chart('Matt Phillips') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Matt Phillips.png', transparent=True, dpi=72)

percentile_chart('Romaine Sawyers') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Romaine Sawyers.png', transparent=True, dpi=72)

percentile_chart('Robert Snodgrass') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Robert Snodgrass.png', transparent=True, dpi=72)

percentile_chart('Okay Yokuşlu') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Bromwich Albion\MF\Okay Yokuşlu.png', transparent=True, dpi=72)

In [ ]:
#West Ham

percentile_chart('Manuel Lanzini') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\MF\Manuel Lanzini.png', transparent=True, dpi=72)

percentile_chart('Mark Noble') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\MF\Mark Noble.png', transparent=True, dpi=72)

percentile_chart('Declan Rice') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\MF\Declan Rice.png', transparent=True, dpi=72)

percentile_chart('Tomáš Souček') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\MF\Tomáš Souček.png', transparent=True, dpi=72)

percentile_chart('Jesse Lingard') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\West Ham\MF\Jesse Lingard.png', transparent=True, dpi=72)

In [ ]:
#Wolves

percentile_chart('Leander Dendoncker') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\MF\Leander Dendoncker.png',transparent=True, dpi=72)

percentile_chart('João Moutinho') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\MF\João Moutinho.png', transparent=True, dpi=72)

percentile_chart('Rúben Neves') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\MF\Rúben Neves.png', transparent=True, dpi=72)

percentile_chart('Owen Otasowie') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\MF\Owen Otasowie.png', transparent=True, dpi=72)

percentile_chart('Vitinha') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\MF\Vitinha.png', transparent=True, dpi=72)

percentile_chart('Daniel Podence') 
plt.savefig(r'C:\Users\Marti\OneDrive\Skrivebord\Prototype\Frontend\Angular\src\assets\PL_DataBase\Wolverhampton\FW\Daniel Podence.png', transparent=True, dpi=72)

In [ ]:
def directory(squad_name):

    players = passing_percentiles['squad'] == squad_name
    return passing_percentiles[players] 

In [ ]:
directory('Arsenal')